<a href="https://colab.research.google.com/github/xipengchen/Personal-Application-Python/blob/master/%E9%95%BF%E7%99%BD%E5%B1%B1_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding:utf-8 -*-

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.

In [3]:
from huggingface_hub import login
login()


In [4]:
from datasets import load_dataset
data_name = 'Jasper0328/changbai'
dataset = load_dataset(data_name,split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['output', 'instruction', 'input'],
    num_rows: 1860
})

In [6]:
dataset = dataset.map(lambda example: {'text': example['instruction'] + example['output']})

Map:   0%|          | 0/1860 [00:00<?, ? examples/s]

In [7]:
dataset[0]

{'output': '在吉林市江南乡阿什哈达村、吉丰铁路遂道南侧临江绝壁上，有两块摩崖刻石，通称阿什哈达摩崖碑。它是研究明朝经略东北的重要历史遗迹。阿什哈达摩崖刻石的存在，证明了明朝政府曾在松花江阿什哈达一带开设船厂，制造“巨舡”。其督军造船的直接目的是为了与黑龙江口的奴儿干都司相联络，用于载军、运粮、转输赏赉和贡赋。阿什哈达摩崖刻石的存在，证明早在十五世纪，东北的辽阔疆土就为明朝政府所统辖。为了加强对摩崖碑的保护，1983年由吉林省文化厅拨款，吉林市博物馆监修，在第一碑处修建一阁，第二碑处修建一亭。',
 'instruction': '请帮我介绍一下阿什哈达摩崖.',
 'input': '阿什哈达摩崖',
 'text': '请帮我介绍一下阿什哈达摩崖.在吉林市江南乡阿什哈达村、吉丰铁路遂道南侧临江绝壁上，有两块摩崖刻石，通称阿什哈达摩崖碑。它是研究明朝经略东北的重要历史遗迹。阿什哈达摩崖刻石的存在，证明了明朝政府曾在松花江阿什哈达一带开设船厂，制造“巨舡”。其督军造船的直接目的是为了与黑龙江口的奴儿干都司相联络，用于载军、运粮、转输赏赉和贡赋。阿什哈达摩崖刻石的存在，证明早在十五世纪，东北的辽阔疆土就为明朝政府所统辖。为了加强对摩崖碑的保护，1983年由吉林省文化厅拨款，吉林市博物馆监修，在第一碑处修建一阁，第二碑处修建一亭。'}

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


model_name = "TinyPixel/Llama-2-7B-bf16-sharded"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [10]:
from peft import LoraConfig, get_peft_model

lora_alpha = 64
lora_dropout = 0.1
lora_r = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM")


In [11]:
!pip install -q wandb -U

import wandb, os
wandb.login()

wandb_project = "changbaiGPT-finetune"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [13]:

from transformers import TrainingArguments

project = "ChangbaiGPT"
base_model_name = "llama2-7b"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 100
num_train_epochs = 4
logging_steps = 50
learning_rate = 2.5e-5
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "linear"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    report_to="wandb",
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [14]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/1860 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [15]:

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [62]:
torch.cuda.empty_cache()

In [63]:
model.config.use_cache = False

In [16]:
trainer.train()

wandb: Currently logged in as: xchen9. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,1.820300
100,1.728200
150,1.603700
200,1.618400
250,1.577100
300,1.579400
350,1.491000
400,1.584600
450,1.488200
500,1.592200


TrainOutput(global_step=500, training_loss=1.6083096008300781, metrics={'train_runtime': 401.6184, 'train_samples_per_second': 2.49, 'train_steps_per_second': 1.245, 'total_flos': 1.0243169503838208e+16, 'train_loss': 1.6083096008300781, 'epoch': 0.54})

In [28]:
model.save_pretrained("trained-model")
model.push_to_hub("Jasper0328/Changbai_Llama2_Finetuned",create_pr=1)

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jasper0328/Changbai_Llama2_Finetuned/commit/091afb4d831a2d1d512d78cf530b0296d8fc6ddd', commit_message='Upload model', commit_description='', oid='091afb4d831a2d1d512d78cf530b0296d8fc6ddd', pr_url='https://huggingface.co/Jasper0328/Changbai_Llama2_Finetuned/discussions/1', pr_revision='refs/pr/1', pr_num=1)

In [21]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [22]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

## 测试 模型


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "TinyPixel/Llama-2-7B-bf16-sharded"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Llama 2 7B, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [7]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "llama2-7b-ChangbaiGPT/checkpoint-300")

In [8]:
eval_prompt = " 请帮我介绍一下阿什哈达摩崖"
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=300)[0], skip_special_tokens=True))

RuntimeError: ignored

## 试错代码如下

In [25]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [30]:
eval_prompt = " 请帮我介绍一下吉林文庙 "
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=300)[0], skip_special_tokens=True))

RuntimeError: ignored

In [26]:
text = "请帮我介绍一下吉林文庙"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


RuntimeError: ignored